In [1]:
import PEPit
from PEPit.functions import SmoothStronglyConvexFunction, StronglyConvexFunction, ConvexFunction, SmoothConvexFunction
from PEPit.primitive_steps import proximal_step
from PEPit.constraint import Constraint

import numpy as np
import cvxpy as cp

# $f(x)=f_1(x_1)+f_2(x_2), \quad g(z)=0$

In [8]:
# verification mode: PEP
problem = PEPit.PEP()

rho = 0.5

L_smooth = 1; mu = 0.1
# f = problem.declare_function(StronglyConvexFunction, mu=mu)
# g = problem.declare_function(StronglyConvexFunction, mu=mu)
f = problem.declare_function(SmoothStronglyConvexFunction, mu=mu, L=L_smooth)
g = problem.declare_function(SmoothStronglyConvexFunction, mu=mu, L=L_smooth)
# f = problem.declare_function(ConvexFunction)
# g = problem.declare_function(ConvexFunction)
func = f + g
x_star, y_star, func_star = func.stationary_point(return_gradient_and_function_value=True)
y1_star, _ = f.oracle(x_star)
y2_star = y_star - y1_star
z1 = problem.set_initial_point()
y1_1 = problem.set_initial_point()
y2_1 = - y1_1

x1_2, grad_f_2, f_2 = proximal_step(z1 - (1/rho) * y1_1, f, 1/rho)
x2_2, grad_g_2, g_2 = proximal_step(z1 - (1/rho) * y2_1, g, 1/rho)
z2 = (x1_2 + x2_2) / 2
y1_2 = y1_1 - rho * (z2 - x1_2)
y2_2 = y2_1 - rho * (z2 - x2_2)

E_1 = (1/rho) * (y1_1 + y1_star)**2 + (1/rho) * (y2_1 + y2_star)**2 + 2 * rho * (z1 - x_star)**2 
E_2 = (1/rho) * (y1_2 + y1_star)**2 + (1/rho) * (y2_2 + y2_star)**2 + 2 * rho * (z2 - x_star)**2 
Delta_1 = rho * (x1_2 - z2)**2 + rho * (x2_2 - z2)**2 + rho * (z1 - z2)**2 

problem.set_performance_metric(E_2 - (E_1 - Delta_1))

diff = problem.solve(verbose=0, solver=cp.MOSEK)
print(f"{diff=}")

diff=-1.2122119865419926e-11


# $f(x)=f_1(x_1), \quad g(z)=f_2(x_2)$

In [13]:
# verification mode: PEP
problem = PEPit.PEP()

rho = 1

L_smooth = 1; mu = 0.001
# f = problem.declare_function(StronglyConvexFunction, mu=mu)
# g = problem.declare_function(StronglyConvexFunction, mu=mu)
# f = problem.declare_function(SmoothStronglyConvexFunction, mu=mu, L=L_smooth)
# g = problem.declare_function(SmoothStronglyConvexFunction, mu=mu, L=L_smooth)
f = problem.declare_function(ConvexFunction)
g = problem.declare_function(ConvexFunction)
func = f + g
x_star, y_star, func_star = func.stationary_point(return_gradient_and_function_value=True)
z0 = problem.set_initial_point()
y0 = problem.set_initial_point()

x1, grad_f_1, f_1 = proximal_step(z0 - (1/rho) * y0, f, 1/rho)
z1, grad_g_1, g_1 = proximal_step(x1 + (1/rho) * y0, g, 1/rho)
y1 = y0 + rho * (x1 - z1)

x2, _, f_2 = proximal_step(z1 - (1/rho) * y1, f, 1/rho)
z2, _, g_2 = proximal_step(x2 + (1/rho) * y1, g, 1/rho)
y2 = y1 + rho * (x2 - z2)
problem.add_constraint(Constraint(func_star - f_2 - g_2 - y_star * (x2 - z2), equality_or_inequality="inequality"))

E_1 = (1/rho) * (y1 - y_star)**2 + rho * (z1 - x_star)**2
E_2 = (1/rho) * (y2 - y_star)**2 + rho * (z2 - x_star)**2
Delta_1 = rho * (x2 - z2)**2 + rho * (z2 - z1)**2

problem.set_performance_metric(E_2 - (E_1 - Delta_1))

diff = problem.solve(verbose=0, solver=cp.MOSEK)
print(f"{diff=}")

diff=-2.1470603073225902e-12
